#  Code for 2D staggered Lagrangian method in Cartersian coordinates

## Control equations

一般形式下，弹塑性流体控制方程为：

质量方程：
\begin{equation}
  \frac{dm}{dt}=0
  \end{equation}
动量方程：
\begin{equation}
  \frac{d\rho \mathbf{u}}{dt}= \nabla \cdot  \mathbf{\Pi} 
\end{equation}

能量方程：
\begin{equation}
 \rho \frac{de}{dt}= \mathbf{\sigma} \cdot \mathbf{\varepsilon}
\end{equation}

\section{二维控制方程}

运动方程：
\begin{equation}
  \frac{d(r,z)}{dt}=(u,v)
\end{equation}

质量守恒：
\begin{equation}\label{eq:38}
  \frac{dm}{dt}=0
\end{equation}

动量守恒方程：
\begin{equation}\label{eq:3}
  \begin{array}{l}
	\displaystyle \rho \frac{du}{dt}=\frac{\partial \sigma _{rr}}{\partial r}+ \frac{\partial s_{rz}}{\partial z}\\[0.3cm]
  \displaystyle \rho \frac{dv}{dt}=\frac{\partial \sigma _{zz}}{\partial z}+ \frac{\partial s_{rz}}{\partial r}\\
  \end{array}
  \end{equation}

能量守恒方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \rho \frac{de}{dt}=\sigma _{rr}\frac{\partial u}{\partial r}+\sigma_{zz}\frac{\partial v}{\partial z}+s_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle = -p\frac{dV}{dt}+ s_{rr}\frac{\partial u}{\partial r}+s_{zz}\frac{\partial v}{\partial z}+s_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)\\
	\end{array}
	\end{equation}

本构方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \frac{ds_{rr}}{dt}=2\mu \left(\frac{\partial u}{\partial r}-\frac{1}{3}\nabla \cdot \vec{u} \right) +s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{zz}}{dt}=2\mu \left(\frac{\partial v}{\partial z}-\frac{1}{3}\nabla \cdot \vec{u} \right) -s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{rz}}{dt}=\mu \left(\frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right) + \frac{s_{rr}-s_{zz}}{2}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\
  \end{array}
  \end{equation}

Von Mises屈服条件：
\begin{equation}
  \frac{3}{2}(s_{rr}^2+s_{zz}^2+s_{\theta \theta}^2+2s_{xy}^2)\le (Y^0)^2
\end{equation}







$$ s_{\theta\theta} = - (s_{rr} +s_{zz}) $$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

<img src="cellpoint.png" width = "800" height = "400" div align=center />

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

In [1]:
struct Const
    Y0 ::Float64 #Yielding strength
    ρ0 ::Float64 
    Γ0 ::Float64
    μ  ::Float64
    a0 ::Float64
    s0 ::Float64
end


In [2]:
mutable struct Var
    r::Array{Float64,1}
    z::Array{Float64,1}
    u::Array{Float64,1}
    v::Array{Float64,1}
    ρ::Array{Float64,1}
    p::Array{Float64,1}
    s::Array{Float64,2} # σ[1] = σrr σ[2] = σzz σ[3] = σrz
end    

In [3]:
struct prb
    tt::Float64
    sf::Float64
end

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [187]:
function MassAndDens(mz,x,y,xc)
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    ρAp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Az=area_quadrangle(xz,yz)
        Az1=0
        ν == 1 ? Az1=volume(xz,yz) : Az1=Az
        ρ[i] = mz[i]/Az1
        for j = 1:4
            ip = Tp[i,j]
            ρAp[ip]  +=  Az*ρ[i]/4
        end
    end
  #@show ρ,ρAp
    return ρ,ρAp 
    
end

MassAndDens (generic function with 1 method)


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [5]:
function A_z(xz,yz) 
    A =zeros(Float64,4)
    xc= sum(xz[i] for i in 1:4)/4
    yc= sum(yz[i] for i in 1:4)/4
    
    A41 = Area3(xz[4],xz[1],xc,yz[4],yz[1],yc)
    A12 = Area3(xz[1],xz[2],xc,yz[1],yz[2],yc)
    A23 = Area3(xz[2],xz[3],xc,yz[2],yz[3],yc)
    A34 = Area3(xz[3],xz[4],xc,yz[3],yz[4],yc)

#     A[1] = (5A41+5A12+A23+A34)/12
#     A[2] = (A41+5A12+5A23+A34)/12
#     A[3] = (A41+A12+5A23+5A34)/12
#     A[4] = (5A41+A12+A23+5A34)/12
    
   A[1] = (A41+A12+A23+A34)/4
   A[2] = (A41+A12+A23+A34)/4
   A[3] = (A41+A12+A23+A34)/4
   A[4] = (A41+A12+A23+A34)/4
    
    return A
end

A_z (generic function with 1 method)

<img src="n12.png" width = "300" height = "300" div align=center />

$$\overrightarrow{n}_{12} = \frac{1}{L_{12}}(y_2- y_1, x_1-x_2)  $$

$$\overrightarrow{\tau}_{12} = \frac{1}{L_{12}}(x_2- x_1, y_2-y_1)  $$

$$ \mathbf{F_1}  = \frac{1}{2}  \left[ -\sigma_{rr}(z_2-z_4)+ s_{rz}(r_2 -r_4), \sigma_{zz}(r_2-r_4) +s_{rz}(z_4 -z_2) , \right] $$ 

In [182]:
function force_zone(x,y,u,v,ρ,p,σ,xc,mz)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
   # Az = A_z(x,y)
    
    for i in 1:4
        i == 1 ? im = 4 : im = i-1
        i == 4 ? ip = 1 :  ip = i+1
        Fx[i] = 0.5(σ[1]*(y[im] - y[ip])+ σ[3]*(x[ip] - x[im])) # + ν*Az[i]*(2σ[1]+σ[2]+3*p)/xc #*sign(-y[im]+y[ip])
        Fy[i] = 0.5(σ[3]*(y[im] - y[ip])+ σ[2]*(x[ip] - x[im])) #+ ν*Az[i]*(σ[3])/xc #*sign(-x[ip]+x[im])
    end
      
    ex,ey = hourglass_viscosity(x, y, u, v, ρ, p,σ) 

    Fx = Fx .+ ex
    Fy = Fy .+ ey
    
    return Fx,Fy
end           

force_zone (generic function with 1 method)

In [7]:
sign(2)

1

\begin{equation}
  \begin{aligned}
	&\displaystyle \rho \frac{de}{dt}= \mathbf{\sigma} \cdot \mathbf{\varepsilon}\\ 
	&=
	\sigma _{rr}\frac{\partial u}{\partial r}+\sigma_{zz}\frac{\partial v}{\partial z}+\sigma_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)
	\end{aligned}
	\end{equation}
    
\begin{equation}
  \begin{aligned}
	&\displaystyle M \frac{de}{dt}=  
\int_{\Omega} \sigma _{rr}\frac{\partial u}{\partial r}+\sigma_{zz}\frac{\partial v}{\partial z}+\sigma_{rz}\left( \frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right)d\Omega
	\end{aligned}
	\end{equation}


\begin{equation}
  \begin{aligned}
\int_{\Omega}  \frac{\partial u}{\partial r} d\Omega&=  -\oint_{\partial \omega}u dz\\
  &= \frac{1}{2} \left((u_1-u_3)(z_2-z_4)+(u_2-u_4)(z_3-z_1)\right)
  \end{aligned}
\end{equation}

\begin{equation}
  \begin{aligned}
\int_{\Omega}  \frac{\partial u}{\partial z} d\Omega&= \oint_{\partial \omega}u dy\\
  &= -\frac{1}{2} \left((u_1-u_3)(r_2-r_4)+(u_2-u_4)(r_3-r_1)\right)
  \end{aligned}
\end{equation}

$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2A}((u_1-u_3)(y_2-y_4)+(u_2-u_4)(-y_1+y_3))$$ 

$$ \frac{\partial v}{\partial y}  \approx -\frac{1}{2A}((v_1-v_3)(x_2-x_4)+(v_2-v_4)(-x_1+x_3))$$ 

对于节点
$$ \rho \frac{d \mathbf{u}}{dt} = - \nabla p$$

$$m_p \frac{d\bf{u}_p}{dt} = \sum_{p\in P(z)} \bf{F}_z^p$$

$$F_z^1 = \mathbf{F}_1 + c\mathbf{e}_1$$

本构方程：
\begin{equation}
  \begin{array}{l}
	\displaystyle \frac{ds_{rr}}{dt}=2\mu \left(\frac{\partial u}{\partial r}-\frac{1}{3}\nabla \cdot \vec{u} \right) +s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{zz}}{dt}=2\mu \left(\frac{\partial v}{\partial z}-\frac{1}{3}\nabla \cdot \vec{u} \right) -s_{rz}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\[0.3cm]
	\displaystyle \frac{ds_{rz}}{dt}=\mu \left(\frac{\partial u}{\partial z}+\frac{\partial v}{\partial r}\right) - \frac{s_{rr}-s_{zz}}{2}\left(\frac{\partial u}{\partial z}-\frac{\partial v}{\partial r}\right)\\
  \end{array}
  \end{equation}

Von Mises屈服条件：
\begin{equation}
  \frac{3}{2}(s_{rr}^2+s_{zz}^2+s_{\theta \theta}^2+2s_{xy}^2)\le (Y^0)^2
\end{equation}


In [342]:
function rhs(var,mz,rc,ρAp)
    μ = con1.μ
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ic, = size(Tp)
    Ip, = size(Tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip) 
    rhse = zeros(Float64,Ic)
    rhss = zeros(Float64,Ic,3)
    σ = zeros(Float64,Ic,3)
  
    σ[:,1] = s[:,1] - p[:]
    σ[:,2] = s[:,2] - p[:]
    σ[:,3] = s[:,3] 
    
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
    Fxn = zeros(Float64,Ip,4)
   # Fxn = zeros(Float65,Ip,4)
    nc = zeros(Int,4)
    for i in 1:Ic
        for j =1:4
            rcell[j] = r[Tp[i,j]]
            zcell[j] = z[Tp[i,j]]
            ucell[j] = u[Tp[i,j]]
            vcell[j] = v[Tp[i,j]]
        end
        uc = sum(ucell)/4
        
        Az = area_quadrangle(rcell,zcell)
        Fx,Fy = force_zone(rcell,zcell,ucell,vcell,ρ[i],p[i],σ[i,1:3],rc[i],mz[i])  
        ∂u_∂r,∂u_∂z = ∂uv(rcell,zcell, ucell)
        ∂v_∂r,∂v_∂z = ∂uv(rcell,zcell, vcell)
        #rhse[i] = 1/(ρ[i]*Az)*(σ[i,1]*∂u_∂r + σ[i,2]*∂v_∂z+σ[i,3]*(∂u_∂z + ∂v_∂r))
             #       + ν*(-σ[i,1]-σ[i,2]-3p[i])*uc/(ρ[i]*rc[i])
         
        ∇u = ∂u_∂r + ∂v_∂z+ uc/rc[i]*ν*Az
       
        rhse[i] = (-p[i]*∇u+s[i,1]*∂u_∂r+s[i,2]*∂v_∂z+
                (-s[i,1]-s[i,2])*uc/rc[i]*ν*Az+σ[i,3]*(∂u_∂z + ∂v_∂r))/(ρ[i]*Az)
        for j = 1:4
            ip = Tp[i,j]
            rhsu[ip] +=  Fx[j]/ρAp[ip] + ν*(2s[i,1]+s[i,2])/(4rc[i]*ρ[i])
            rhsv[ip] +=  Fy[j]/ρAp[ip] + ν*(s[i,3])/(4rc[i]*ρ[i])
            
        end  
       
        
    #   Fxn[i,:]= Fx[:]/ρAp
        
    
        rhss[i,1] =  (2μ*(∂u_∂r - 1/3*∇u) + σ[i,3]*(∂u_∂z - ∂v_∂r)) /Az
        rhss[i,2] =  (2μ*(∂v_∂z - 1/3*∇u) - σ[i,3]*(∂u_∂z - ∂v_∂r)) /Az
        rhss[i,3] =   (μ*(∂u_∂z+∂v_∂r)  +   (-σ[i,1] + σ[i,2])*(∂u_∂z - ∂v_∂r)) /Az    
        
#          rhss[i,1] =  2μ*(∂u_∂r - 1/3*∇u)  ./Az
#         rhss[i,2] =  2μ*(∂v_∂z - 1/3*∇u)  ./Az
#         rhss[i,3] =   μ*(∂u_∂z+∂v_∂r)  ./Az    
    end
    
    @show Fxn[1,2] + Fxn[2,1],Fxn[1,3] + Fxn[2,4],rhsu
    # rhsu = rhsu./ρAp
  #   rhsv = rhsv./ρAp
    
#     for i in 1:Ip
#         nc[1:4] .= Tc[i,1:4]
#         for j in 1:4
#             if nc[j]>0
#                 rhsu[i] += (s[nc[j],1]-s[nc[j],2])/(4rc[nc[j]]*ρ[nc[j]])
#                 rhsv[i] += (s[nc[j],3])/(4rc[nc[j]]*ρ[nc[j]])
#             end
#         end
#     end
    
    return rhse, rhsu,rhsv,rhss

end     

rhs (generic function with 2 methods)

In [9]:
function output(f)
    io = open("data2.dat", "w+")
     writedlm(io,f,"  ") 
    close(io)
end

output (generic function with 1 method)

$$ \mathbf{S} = \mathbf{S} \times \text{min} \left(1,Y_0/\sqrt{\frac{3}{2}\mathbf{S}:\mathbf{S}}\right)$$

In [10]:
function yield_s!(s,con)
    Y0 = con.Y0
    I, = size(s)
    for i in 1:I
         sums = s[i,1]^2 +s[i,2]^2 +2*s[i,3]^2 +(s[i,1]+s[i,2])^2
        for j = 1:3
       
        s[i,j] = s[i,j] * min(1,Y0/√(3/2*sums))
            end 
    end
    return s
end

yield_s! (generic function with 1 method)

In [297]:
function predictor_corrector2nd(dt,var::Var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ip, = size(u)
    Ic, = size(p)
    
    rc = r_c(r,z)
     
    ρ,ρAp = MassAndDens(mz,r,z,rc)
    e = p_to_e(ρ, p,con1)    
  #  p = artificial_viscosity!(r,z,u,v,ρ,p,s,rc)
    ρ,p,s,rc = bound_ghost_cell!(ρ,p,s,rc)
    var = Var(r, z, u, v, ρ, p, s)
    rhse, rhsu,rhsv,rhss = rhs(var,mz,rc,ρAp)
    rhsu,rhsv = bound_force(rhsu,rhsv,var)
    
   # s = yield_s!(s,con1)
    
    r =  r + dt*u
    z  = z + dt*v
    u  = u + dt*rhsu
    v  = v + dt*rhsv
    e = e + dt*rhse
    s = s + dt*rhss
    #@show z₀    
     s= yield_s!(s,con1)
    
#     r₀ =  r + dt*u
#     z₀  = z + dt*v
#     u₀  = u + dt*(rhsu ./ ρAp)
#     v₀  = v + dt*(rhsv ./ ρAp)
#     e₀  = e + dt*rhse
#     s₀  = s + dt*rhss
#     #@show z₀    
#      s₀ = yield_s!(s₀,con1)
    
#     r₀,z₀,u₀,v₀ = bound_ghost_coordinate!(r₀,z₀,u₀,v₀)
#      rc = r_c(r₀,z₀)
#     ρ,ρAp = MassAndDens(mz,r₀,z₀,rc)
#     p = e_to_p(ρ,e₀,con1)
#     p = artificial_viscosity!(r₀,z₀,u₀,v₀,ρ,p,s,rc) 
#     ρ,p,s₀,rc = bound_ghost_cell!(ρ,p,s₀,rc)
#     var = Var(r₀, z₀, u₀, v₀, ρ, p, s₀)
   
#     rhse₀,rhsu₀,rhsv₀,rhss₀ = rhs(var,mz,rc)
#     rhsu₀,rhsv₀ = bound_force(rhsu₀,rhsv₀,var)
#   #  @show s[:,1]
#     r +=  0.5dt*(u + u₀)
#     z +=  0.5dt*(v + v₀)
#     u +=  0.5dt*(rhsu + rhsu₀) ./ ρAp
#     v +=  0.5dt*(rhsv + rhsv₀) ./ ρAp
#     e +=  0.5dt*(rhse + rhse₀)
#     s  += 0.5dt*(rhss + rhss₀)
    
#     s = yield_s!(s,con1)
    
    
    r,z,u,v = bound_ghost_coordinate!(r,z,u,v)
    rc = r_c(r,z)
 #   @show r 
    ρ,ρAp = MassAndDens(mz,r,z,rc)
    
    p = e_to_p(ρ,e,con1)
    ρ,p,s,rc = bound_ghost_cell!(ρ,p,s,rc)
    var = Var(r,z,u,v,ρ,p,s)
  # @show r,z
   # println(rc,ρAp)
    return var,ρAp,rc

end

predictor_corrector2nd (generic function with 1 method)

In [206]:
function r_c(r,z)
    Ic, = size(Tp)
    Ib, = size(Tb)
    rc = zeros(Float64,Ic)
       for i in 1:Ic
    #    rc[i] = sum(r[Tp[i,1:4]])/4
        rc[i] = volume(r[Tp[i,1:4]],z[Tp[i,1:4]])/area_quadrangle(r[Tp[i,1:4]],z[Tp[i,1:4]])
    end

        for i in 1:Ib
    if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            rc[ic] = rc[ic2] 
          end 
    end
    return rc 
end


r_c (generic function with 2 methods)

In [13]:
function fη(ρ,c::Const)
    η = ρ/c.ρ0
    fη=(η .-1.0) .* (η .-c.Γ0*(η .-1.0)/2.0) ./ (η .-c.s0*(η .-1)) .^2
end

function fηη(ρ,c::Const)
    η = ρ/c.ρ0
    fηη=(η .+(c.s0-c.Γ0) .* (η .-1)) ./(η .-c.s0*(η .- 1)) .^3
end

fηη (generic function with 1 method)

In [14]:
function euler1st(dt,var::Var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,r,z)
    e = p_to_e(ρ, p,con1)    
    p = artificial_viscosity!(r,z,u,v,ρ,p)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r, z, u, v, ρ, p, s)
    
    rhse, rhsu,rhsv,rhss = rhs(var)
    rhsu,rhsv = bound_force(rhsu,rhsv,var)
    
 #   @show rhsv
    s = yield_s!(s,con1)
    r =  r + dt*u
    z = z + dt*v
    u  = u + dt*(rhsu ./ mp)
    v  = v + dt*(rhsv ./ mp)
    e  = e + dt*(rhse ./ mz) 
    s  = s + dt*rhss
  #  @show rhsu
    s = yield_s!(s,con1)
    r,z,u,v = bound_ghost_coordinate!(r,z,u,v)
    ρ,mp = MassAndDens(mz,r,z)
    p = e_to_p(ρ,e,con1)
    ρ,p,s = bound_ghost_cell!(ρ,p,s)
    var = Var(r,z,u,v,ρ,p,s)
   
    return var
end

euler1st (generic function with 1 method)

### Boundary force

If one edge of a cell is a free boundary, then $f$ on the points is zero. As we have add it in RHs(), we need to remove it by resolve it again. Different from RHs(), at every point of a cell the force is composited by two parts on different edges. This is distinguished by $k$ when $k = 1$ the boundary is on y direction, forces on 12 and 34 boundary must be subtracted. When $ k =2$ the boundary is on x direction, subtract forces on 41, 23. 

In [205]:
function bound_force(rhsu,rhsv,var)
    Ip, = size(Tc)
    Ib, = size(Tb)
   
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
    ip1 = zeros(Int,4)
    σ = zeros(Float64,3)
    for i in 1:Ib
        if Tb[i,4] == 1 || Tb[i,4] == 3
            ic = Tb[i,1]        
            rcell[1:4] = r[Tp[ic,1:4]]
            zcell[1:4] = z[Tp[ic,1:4]]
            ucell[1:4] = u[Tp[ic,1:4]]
            vcell[1:4] = v[Tp[ic,1:4]]

            σ[1] = s[ic,1] - p[ic]
            σ[2] = s[ic,2] - p[ic]
            σ[3] = s[ic,3]

            ex,ey = hourglass_viscosity(rcell, zcell, ucell, vcell, ρ[ic]
                                        , p[ic],s[ic,:])
            i₁ = Tb[i,3]
            i₂ = i₁+1
            if i₁ == 4
                i₂ = 1
            end
            nx = zcell[i₂] -zcell[i₁]
            ny = rcell[i₁] -rcell[i₂]
            p0 = 1e-5
            for j in (i₁,i₂)

                ip = Tp[ic,j]
                    Fx = -σ[1]*nx/2 -σ[3]*ny/2+ex[j] #*nx^2/(nx^2+ny^2)  #+ex[j] 
                    Fy = -σ[3]*nx/2 -σ[2]*ny/2+ey[j] #*ny^2/(nx^2+ny^2)
                    Fx1 = (Fx*nx+Fy*ny)*nx/(nx^2+ny^2)
                    Fy1 = (Fx*nx+Fy*ny)*ny/(nx^2+ny^2)
                    Fx = Fx- Fx1
                    Fy = Fy - Fy1
                    Fx1 =  p0*nx/2 #+ex[j]*nx^2/(nx^2+ny^2) 
                    Fy1 =  p0*ny/2 #+ey[j]*ny^2/(nx^2+ny^2)

         #           rhsu[ip] +=  -Fx1 # -Fx1
         #           rhsv[ip] +=  -Fy1 #-Fy1
                #
      #              rhsu[ip] = 0
       #             rhsv[ip] = 0

                if Tb[i,4] == 3 # boundary
                    uL = 2e-3
                    u[ip] = uL #*nx/(nx^2 +ny^2)
                end  
            end
        end
    end 

    return rhsu, rhsv
end  

bound_force (generic function with 1 method)

<img src="bound_corresponding.png" width = "400" height = "800" div align=center /> 

$$ Ig_1 = Tb[i,3] $$
$$Ib_1 = Ig_1 -1$$

In [333]:
function bound_ghost_coordinate!(r,z,u,v)
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)
   # r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
   
    rt= copy(r)
    zt= copy(z)
   # output(r)
    for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            ig₁ = Tb[i,3]
            
           
            ig₁ == 4 ? ig₂ = 1 : ig₂ = ig₁+1
            ig₂ == 4 ? ig₃ = 1 : ig₃ = ig₂+1
            ig₃ == 4 ? ig₄ = 1 : ig₄ = ig₃+1

            ig₁ == 1 ? ib₁ = 4 : ib₁ = ig₁-1
            ib₁ == 1 ? ib₂ = 4 : ib₂ = ib₁-1
            ib₂ == 1 ? ib₃ = 4 : ib₃ = ib₂-1
            ib₃ == 1 ? ib₄ = 4 : ib₄ = ib₃-1

                r1 =    rt[Tp[ic2,ib₁]]
                r2 =    rt[Tp[ic2,ib₂]]
                z1 =    zt[Tp[ic2,ib₁]]
                z2 =    zt[Tp[ic2,ib₂]]

                r3=     rt[Tp[ic2,ib₃]]
                z3=     zt[Tp[ic2,ib₃]]
                r4=     rt[Tp[ic2,ib₄]]
                z4=     zt[Tp[ic2,ib₄]]
                
                nx = zt[Tp[ic2,ib₁]] -zt[Tp[ic2,ib₂]]
                ny = rt[Tp[ic2,ib₂]] -rt[Tp[ic2,ib₁]]
                 for j in (ib₁,ib₂)
                     ip = Tp[ic2,j]
                    u[ip] = 0 # u[ip]*ny^2/(nx^2 +ny^2)
              #     v[ip] = v[ip]*nx^2/(nx^2 +ny^2)
                 end

                r[Tp[ic,ig₃]],z[Tp[ic,ig₃]] = reflect(r1,z1,r2,z2,r3,z3)
                r[Tp[ic,ig₄]],z[Tp[ic,ig₄]] = reflect(r1,z1,r2,z2,r4,z4)
                u[Tp[ic,ig₃]],v[Tp[ic,ig₃]] =u[Tp[ic2,ib₃]],-v[Tp[ic2,ib₃]]
                u[Tp[ic,ig₄]],v[Tp[ic,ig₄]] = u[Tp[ic2,ib₄]],-v[Tp[ic2,ib₄]]
            
       
          end 
    end
    return r,z,u,v
end  

bound_ghost_coordinate! (generic function with 1 method)

In [341]:
function bound_ghost_cell!(ρ,p,s,rc)
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)    
    for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            ρ[ic] = ρ[ic2]
            p[ic] = p[ic2]
            s[ic,1:2] = s[ic2,1:2]
            s[ic,3] = -s[ic2,3]
          #  s[ic2,3]=0
          end 
    end
    return ρ,p,s,rc
end  

bound_ghost_cell! (generic function with 1 method)

## Ghost cell coordinate of a reflect boundary

<img src="reflect.png" width = "200" height = "400" div align=center /> 

$$ k_1 k_2 = -1$$
$$ d_3 = -d_4 $$

$$(z_2-z_1)(z_3-z_4)+(r_3-r_4)(r_2-r_1) = 0$$
$$(z_2-z_1)(r_3-r_1)-(z_3-z_1)(r_2-r_1) = -(z_2-z_1)(r_4-r_1)+(z_4-z_1)(r_2-r_1)$$

In [18]:
# using SymPy
# @vars z1 z2 z3 z4 r1 r2 r3 r4
# exp1 = solve([(z2-z1)*(z3-z4)+(r2-r1)*(r3-r4),(z2-z1)*(r3-r1)-(z3-z1)*(r2-r1)+(z2-z1)*(r4-r1)-(z4-z1)*(r2-r1)],[r4,z4])
# @show exp1[r4],exp1[z4]

In [19]:
function reflect(r1,z1,r2,z2,r3,z3)
    tmp1=(r1*r3-r2*r3+z1*z3-z2*z3) 
    tmp2 = (2r1*z2-r1*z3-2r2*z1+r2*z3+r3*z1-r3*z2)
    tmp3 = ((r1-r2)^2+(z1-z2)^2)
    
    r4 = ((r1-r2)*tmp1 - (z1-z2)*tmp2)/tmp3
    z4 = ((r1-r2)*tmp2 + (z1-z2)*tmp1)/tmp3
    
    return r4,z4
end

reflect (generic function with 1 method)

In [20]:
reflect(1,1,1,0,0.9,0)

(1.1, 0.0)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [21]:
function p_to_e(ρ,p,con)
    c=con
    ei = (p .- c.ρ0*c.a0^2*fη(ρ,c))/(c.ρ0*c.Γ0)
    return ei
end
function e_to_p(ρ,ei,con::Const)
    c=con
    p = c.ρ0*c.Γ0*ei .+ c.ρ0*c.a0^2*fη(ρ,c)
    return p
end

e_to_p (generic function with 1 method)

In [22]:
function CFL(SF,var)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ic, = size(ρ)
    cflmin = 1.e9
      cfl = 0 
    for i in 1:Ic
        cfl =0
        u2max = 0
        u2=0.0
        for j = 1:4
            ip = Tp[i,j]
            u2 = u[ip]^2+v[ip]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = (r[Tp[i,4]] - r[Tp[i,1]])^2+(z[Tp[i,4]]-z[Tp[i,1]])^2
       
        for j=1:3
            dl= (r[Tp[i,j+1]] - r[Tp[i,j]])^2+(z[Tp[i,j+1]]-z[Tp[i,j]])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = sound([ρ[i],u[i],p[i],min(s[i,1],s[i,2])],con1)
        cfl = √(dlmin)/(√(u2max)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
 #   @show cflmin
    dt = cflmin*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [23]:
function sound(uo::Array{Float64,1},con::Const,EoP::Int=1)
    a0,ρ0,Γ0,Y0,μ  = con.a0,con.ρ0,con.Γ0,con.Y0,con.μ
    ρ,uu,p,sxx   = uo[1:4]
    a2  = a0^2*fηη(ρ,con)+p/ρ^2*ρ0*Γ0
    if EoP == 2
        c=sqrt(a2-ρ0/ρ^2*Γ0*sxx)
        return c
    else
       #  @show a2,ρ0/ρ^2,4.0/3*μ/ρ
        c=sqrt(a2-ρ0/ρ^2*Γ0*sxx+4.0/3*μ/ρ)
       
        return c
    end
end

sound (generic function with 2 methods)

In [298]:
function TimeSolve(var,mz)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
    t₁ = tt/100
    t₂ = t₁
#     kinetic_init,energy_init = total_kinetic(var,mz)
#     kinetic = kinetic_init
   # while kinetic > 1e-4*kinetic_init && t<tt
 #   while t<tt
  ρAp = zeros(Float64,size(Tc))
    rc=  zeros(Float64,size(Tp))
for i in 1:2
        dt=CFL(sf,var)
 #       @show dt
        if t+dt>tt
           dt = tt-t
        end
        
        var,ρAp,rc = predictor_corrector2nd(dt,var,mz)
       # var = euler1st(dt,var,mz)
#         kinetic,energy = total_kinetic(var,mz)
         t += dt
        if t > t₁
            println(t)
            t₁ += t₂
#             @show kinetic/kinetic_init,(energy+kinetic)/(kinetic_init+energy_init)
        end
        
    end
    return var,ρAp,rc
end

TimeSolve (generic function with 1 method)

In [25]:
function total_kinetic(var,mz)
    r,z,u,v,ρ,p = var.r,var.z,var.u,var.v,var.ρ,var.p
    Ip, = size(Tc)
    Ic, = size(Tp)
    Ib, = size(Tb)
    k = zeros(Float64,Ip)
    
    kinetic = 0
    energy = 0
     ρ1,mp = MassAndDens(mz, r, z)
    
    for i in 1:Ip
        k[i]= mp[i]*0.5*(u[i]^2+v[i]^2)
    end
    
    
    for i in 1:Ic
        energy += mz[i]*p_to_e(ρ[i], p[i], con1)
    end
    
    for i in 1:Ib
        if Tb[i,4] == 2
            ic= Tb[i,1]
            energy -= mz[ic]*p_to_e(ρ[ic], p[ic], con1)
            
            ig₁ = Tb[i,3]
            ig₁ == 4 ? ig₂ = 1 : ig₂ = ig₁+1
            k[Tp[ic,ig₁]] = 0
            k[Tp[ic,ig₂]] = 0
            end
    end
    kinetic = sum(k)
    return kinetic,energy
end


total_kinetic (generic function with 1 method)

## Artificial Viscosity 

Use the strain rate to construct the artificial viscosity 
$$\frac{ds}{dt} = \frac{\partial u}{\partial x}  + \frac{\partial v}{\partial y} $$


Then the artificial viscosity can be written as
$$ q_w = \left\{ \begin{align}
\alpha L \rho \left( \frac{ds}{dt} \right)^2 +  \beta L^2 \rho c \left|\frac{ds}{dt}\right|, \quad \text{if}  \quad \frac{ds}{dt}< 0,\\
0, \quad \text{if} \quad \frac{ds}{dt}\ge 0. \\
\end{align}
\right.
$$
where $\alpha = 0.6$, $\beta = 2.0 $ and $c$ is the sonic speed.

$L$ is the reference length, and constructed as
$$L = A/L_{\text{max}} $$

$$L_{\text{max}} = \text{max}(L_{13},L_{24})$$


In [139]:
function artificial_viscosity_c!(x,y,u,v,ρ,p,s,rc)
    xc = sum(x[i] for i in 1:4)/4
    yc = sum(y[i] for i in 1:4)/4
    
    ∂u_∂x,tmp = ∂uv(x,y,u)
    tmp,∂v_∂y = ∂uv(x,y,v)
    c = sound([ρ,0.0,p,min(s[1],s[2])],con1)
    Lmax = max(√((x[3] - x[1])^2+(y[3] - y[1])^2) ,√((x[2] - x[4])^2+(y[2] - y[4])^2) )  
    A = area_quadrangle(x, y)
    L = A/Lmax 
    uc = sum(u)/4
    ∇ =  (∂u_∂x + ∂v_∂y)/A+ ν*uc/rc 
    if ∇ > 0
        q = 0
    else
        q = 0.6ρ*L*c*abs(∇) +1.0L^2*ρ*∇^2 
    end
    return p+q
end

artificial_viscosity_c! (generic function with 1 method)

$$b_2 \rho Lc \frac{\partial u}{\partial x} - b_{12}\rho L^2 \left(\frac{\partial u}{\partial x}\right)^2$$

In [27]:
function artificial_viscosity!(r,z,u,v,ρ,p,s,rc)
    I, = size(Tp)
    rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)
   
    for i in 1:I
        for j in 1:4
            ip = Tp[i,j]
            rcell[j] = r[ip]
            zcell[j] = z[ip]
            ucell[j] = u[ip]
            vcell[j] = v[ip]
        end
        p[i] = artificial_viscosity_c!(rcell,zcell,ucell,vcell,ρ[i],p[i],s[i,1:3],rc[i])
    end
    
        return p
        
    end       

artificial_viscosity! (generic function with 1 method)

In [28]:
function l_viscosity(x,y,α)
    xc = sum(x)/4; yc = sum(y)/4
    d = zeros(Float64,4)
    for i in 1:4
    d[i] = distance(x[i],y[i],xc,yc,α)
    end
    A = A_z(x, y)
    L = 2sum(A)/sum(d)
    return L
end

l_viscosity (generic function with 1 method)

#### distance from  Point to line 
Line: $$ (xc -cos \alpha )(x-xc) = (yc-sin \alpha)(y-yc)$$
point: $x,y$

In [29]:
function distance(x,y,xc,yc,α)
    d = abs((xc - cosd(α)) * (x .-xc)- (yc-sind(α))*(y .- yc))/√((xc-cosd(α))^2+(yc-sind(α))^2)
    return d
end

distance (generic function with 1 method)

### 沙漏粘性 Hourglass

$$ \text{hg}_x = u_1 -u_2 + u_3 - u_4$$ 
$$ \text{hg}_x = v_1 -v_2 + v_3 - v_4$$ 

$$ e_{1x} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{1y} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{2x} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{2y} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{3x} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{3y} = -\frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

$$ e_{4x} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_x $$
$$ e_{4y} = \frac{1}{4}q\rho c\sqrt{A}\text{hg}_y $$

In [30]:
function hourglass_viscosity(x,y,u,v,ρ,p,s)
    hgx = u[1] - u[2] + u[3] -u[4]
    hgy = v[1] - v[2] + v[3] -v[4]
    
    ex =zeros(Float64,4)
    ey =zeros(Float64,4)
    A = area_quadrangle(x,y)
    c = sound([ρ,0.0,p,min(s[1],s[2])], con1)
 #   @show c
    q = 0.1 # coefficient of hourglass viscosity 0.01 - 0.5 ?
    ex[1] = -0.25q*ρ*c*√(A)*hgx
    ey[1] = -0.25q*ρ*c*√(A)*hgy
    
    ex[2] = -ex[1]
    ey[2] = -ey[1]
    
    ex[3] = ex[1]
    ey[3] = ey[1]
    
    ex[4] = ex[2]
    ey[4] = ey[2]
    
  ex .= 0
  ey .= 0
 #   @show ex,ey
    return ex, ey
end

hourglass_viscosity (generic function with 1 method)

In [191]:
function area_quadrangle(x::Array{Float64,1},y::Array{Float64,1}) 
    A = 0.5abs( (x[1]-x[3])*(y[2]-y[4])+(x[2]-x[4])*(y[3]-y[1]))
    return A
end

area_quadrangle (generic function with 1 method)

In [32]:
function Area3(x1,x2,x3,y1,y2,y3)
    return abs((x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2)
end

Area3 (generic function with 1 method)

###  $\frac{\partial (u,v)}{\partial (x,y)}$

we use the diffences between 1 and 3 points and 2 and 4 points to solve the partial differences approximately, as
$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2A}((u_1-u_3)(y_2-y_4)+(u_2-u_4)(-y_1+y_3))$$ 

$$ \frac{\partial v}{\partial y}  \approx -\frac{1}{2A}((v_1-v_3)(x_2-x_4)+(v_2-v_4)(-x_1+x_3))$$ 
Similar process to $\frac{\partial u}{\partial y}$,$\frac{\partial v}{\partial x}$ and $\frac{\partial v}{\partial y}$

In [33]:
function ∂uv(x,y,u)
    ∂u_∂x = 0.5(u[1]-u[3])*(y[2]-y[4]) + 0.5(u[2]-u[4])*(y[3]-y[1])
    ∂u_∂y = -0.5(u[1]-u[3])*(x[2]-x[4]) - 0.5(u[2]-u[4])*(x[3]-x[1])
    return ∂u_∂x,∂u_∂y
end

∂uv (generic function with 1 method)

## Output function

In [34]:
function cell_coordinate(x,y)
    Ip, = size(Tc)
    Ic, = size(Tp)
    xc = zeros(Float64,Ic)
    yc = zeros(Float64,Ic)
    
    for i in 1:Ic
        for j = 1:4
            ip = Tp[i,j] 
            xc[i] += x[ip]/4
            yc[i] += y[ip]/4
        end
        
    end
    
    return xc,yc
end

cell_coordinate (generic function with 1 method)

In [35]:
function outputline(Ix,Iy,var,mz)
    r,z,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    I, = size(ρ)
    
    uline =  zeros(Float64,Ix)
    rline =  zeros(Float64,Ix)
    zline =  zeros(Float64,Ix)
    ρline =  zeros(Float64,Ix)        
    pline =  zeros(Float64,Ix)  
    sline = zeros(Float64,Ix,3)
    for i in 1:Ix
        j = Int(floor(Iy/2))
            ip = i+(j-1 ) *(Ix+1)
            ic =i+(j-1)*Ix
            uline[i] = u[ip]
            rline[i]  = r[ip]
            ρline[i]  = ρ[ic]
            pline[i] = p[ic]
           # σline[i,:] .= σ[ic,:]
            sline[i,:] = s[ic,:]
    end
    return rline,uline,ρline,pline,sline
end

outputline (generic function with 1 method)

In [36]:
function Output_cell(Ix,Iy,var,mz)
    x,y,u,v,ρ,p,s= var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data.dat", "w+")
    A = zeros(Float64,Ip,10)
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\" \"srr\" \"szz\" \"szr\" \"s:s\"  ZONE T=\"Zone 1\" 
I=$Ix,J=$Iy,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "

    write(io,text) 
    for i in 1:Ic
        xc = 0
        yc =0
        uc =0
        vc =0
        for j = 1:4
            ip = Tp[i,j] 
                xc += x[ip]/4
                yc += y[ip]/4
                uc += u[ip]/4
                vc += v[ip]/4
        end
        
        A[i,1] = xc
        A[i,2] = yc
        A[i,3] = uc
        A[i,4] = vc
        A[i,5] = ρ[i]        
        A[i,6] = p[i]      
        A[i,7:9] .= s[i,1:3] 
        A[i,10] = (s[i,1]^2+s[i,2]^2 +2s[i,3]^2+(s[i,1]+s[i,2])^2)
    end
      writedlm(io, A, "  ")
    close(io)
end

Output_cell (generic function with 1 method)

In [37]:
function Output_point(Ix,Iy,var,mz)
    x,y,u,v,ρ,p,s = var.r,var.z,var.u,var.v,var.ρ,var.p,var.s
    Ip, = size(Tc)
    Ic, = size(Tp)
    
    io = open("data.dat", "w+")
    #s = zeros(Float64,Ip,3)
    A = zeros(Float64,Ip,9)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\" \"srr\" \"szz\" \"szr\"   ZONE T=\"Zone 1\" 
I=$(Ix+1),J=$(Iy+1),K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)
    
       ρp = zeros(Float64,Ip)
       pp = zeros(Float64,Ip)
       np = zeros(Int,Ip)
       sp = zeros(Float64,Ip,3)
    
    for i in 1:Ic
        
        xc = 0
        yc =0
        uc =0
        vc =0
        for j = 1:4
            ip = Tp[i,j] 
            ρp[ip] += ρ[i]
            pp[ip] += p[i]
            sp[ip,:] += s[i,:]
            np[ip] += 1   
        end
    end
        ρp = ρp ./ np
        pp = pp ./ np
        sp = sp ./ np
    
        A[:,1] = x
        A[:,2] = y
        A[:,3] = u
        A[:,4] = v
        A[:,5] = ρp
        A[:,6] = pp
        A[:,7:9] .= sp[:,1:3]
    
      writedlm(io, A, "  ")
    close(io)
end

Output_point (generic function with 1 method)

## Cases

In [41]:
 function Piston()

    global problem = prb(1.5e2,0.1)
    # global problem = prb(3e-5,0.3)
     
    I = 200
    J = 5
    dx = 100/I
    dy = 10/J
    @show typeof(dx)
    I1= I+1
    J1= J+2
    global   con1 = Const(9e-5,8.930,2.0,4.5e-1,0.3940,1.49)
  #  global   con1 = Const(0.0,8930,2.0,4.5e10,3940.0,1.49)
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1-2,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    

# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 4
            Tb[ib,4] = 3
        end
    
      for i = 1:I1-1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 2
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 2
                end
            end
        end
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 4
             Tb[ib,4] = 2
         end
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            x[ip] = (i-1)*dx
            y[ip] = (j-2)*dy
            u[ip] = 0
            v[ip] = 0.0
        end
    end
    
      
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y = bound_ghost_coordinate!(x,y)
    x,y = bound_ghost_coordinate!(x,y)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                p[ic] = 1.e-7
                ρ[ic] = 8.930
                mz[ic] = ρ[ic] * Az[ic]
                s[ic,:] .= 0       
        end
    end 
    return var,mz,I1,J1
end

Piston (generic function with 1 method)

In [ ]:
var,mz,I,J=Piston()
#Output1
var= TimeSolve(var,mz)

In [ ]:
push!(LOAD_PATH,"/home/bfly/workspace/Juliastudy/Src")

In [ ]:
uo,u,x,inter = Lag1DNum.Piston()
uo,x  = Lag1DNum.TimeSolve(uo, x,inter)

In [ ]:
rcell, zcell = cell_coordinate(var.r,var.z)

In [46]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [179]:
 function Pistonz()

    global problem = prb(1.5e-4,0.3)
    # global problem = prb(3e-5,0.3)
     
    I = 10
    J = 100
    dx = 0.1/I
    dy = 1/J
    
    global   con1 = Const(9e7,8930,2.0,4.5e10,3940.0,1.49)
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ic =i+(j-1)*I
                #ic = Tc[ip]
                Tb[ib,1] = ic
                if i==1 
                    Tb[ib,2] = 4 # 1->4 
                    Tb[ib,3] = 2 # type 1:free 2:
                end   
                 if  i==I+1 
                    Tb[ib,2] = 2 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tb[ib,3] = 2
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tb[ib,3] = 1
                end
                if j==J+1
                    Tb[ib,2] = 3 
                    Tb[ib,3] = 1
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    σ = zeros(Float64,Ic,3)
    
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = (i-1)*dx
            y[ip] = (j-1)*dy
            u[ip] = 0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
    
            Az = area_quadrangle(xz,yz) #[1]+A[2]+A[3]+A[4]
          #  if i>=I/2
               
                p[ic] = 1.e5
                ρ[ic] = 8930
                mz[ic] = ρ[ic] * Az
                σ[ic,1] = -p[ic]
                σ[ic,2] = -p[ic]
                σ[ic,3]= 0 
#             else
#                 mz[ic] = 1.0 * Az[ic]
#                 p[ic] = 1.0
#                 ρ[ic] = 1.0
#             end
        end
    end
    var = Var(x,y,u,v,ρ,p,σ)
    return var,mz,I,J
end

Pistonz (generic function with 1 method)

In [ ]:
var,mz,I,J=Pistonz()
#Output1
var = TimeSolve(var,mz)

In [ ]:
rcell, zcell = cell_coordinate(var.r,var.z)

In [ ]:
plot3D(var.r,var.z,var.v)
#triplot()

In [ ]:
tricontourf(var.r,var.z,var.v)

In [ ]:
? Const

## 单位变换

$$ 1kg/m^3 = 10^{-3} g/cm^3$$
$$1m/s = 10^{-4} cm/\mu s$$
$$ 1Pa = 1N/m^2 = 1kg/(m\cdot s^2) = (1000g)/(100cm \cdot 10^{12} \mu s^2) = 10^{-11} g/(cm \cdot s^2)$$ 

In [97]:
 function Collapse()

    global problem = prb(1.3e2,0.1)
  #  global problem = prb(50,0.3)
    # global problem = prb(5e-5,0.3)
    
    I = 20
    J = 16
    dr = 2.0/J
    dθ = 90/I
    
    I1 = I +2
    J1 = J
    
    global   con1 = Const(3.3e-3,1.845,2.,1.516,1.287,1.124)
  #  global   con1 = Const(3.3e-3,1.845,2.,0,1.287,1.124)
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
     Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    global ν = 1
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    

# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 2
            Tb[ib,4] = 2
        end
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 4
             Tb[ib,4] = 2
         end
      for i = 1:I1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 1
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 1
                end
            end
        end
        
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            r₁ = (j-1)*dr+8
            θ = 90-(i-2)*dθ
            v₀ = 4.171e-2
            x[ip] = cosd(θ)*r₁
            y[ip] = sind(θ)*r₁
            u[ip] = -v₀*8/r₁ *cosd(θ)
            v[ip] = -v₀*8/r₁ *sind(θ)
        end
    end
      
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y,u,v = bound_ghost_coordinate!(x,y,u,v)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            xc= sum(xz)/4
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                p[ic] = 1.e-10
                ρ[ic] = 1.845
                mz[ic] = ρ[ic] * Az[ic]*xc^ν
                s[ic,:] .= 0       
        end
    end 
    Ib, = size(Tb)
     for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            mz[ic] = mz[ic2]
          end 
    end
   # ρ,p,s,rc = bound_ghost_cell!(ρ, p, s, rc)  
    return var,mz,I1,J1
end

Collapse (generic function with 1 method)

In [98]:
var,mz,I,J = Collapse()
var = TimeSolve(var,mz)

1.3052866470150375
2.6047030997579177
3.9074058028098335
5.207092396583464
6.5076614951070795
7.802100121114607
9.10710091208085
10.403985675910217
11.703223071618378
13.00307899493371
14.303020441330158
15.60463140804259
16.90571221770223
18.207141528301925
19.500903921904264
20.80180230631307
22.102765675240057
23.403291749166346
24.702426265732164
26.001036955163098
27.306922607937324
28.602518887366216
29.905483769697533
31.2066372182616
32.50550388264
33.80250040596825
35.105831612488345
36.406348024392344
37.70433188911254
39.008247132008655
40.30878669637917
41.60611343816079
42.90894937364693
44.207848125877405
45.50283865668079
46.80288249060608
48.10758803422472
49.4077561310099
50.703332475477964
52.00314044470614
53.307002616146235
54.60568412147388
55.908211835278976
57.2049717349574
58.50524167346459
59.80892664828968
61.106333743127685
62.40670159567367
63.70053885846594
65.00651855904046
66.3054922351738
67.60685140636744
68.90094231316495
70.206555833179
71.50448821788

Var([-0.484997, 0.0, 0.484997, 0.91198, 1.36997, 1.80547, 2.2417, 2.65171, 3.05856, 3.4335  …  7.14416, 7.52919, 7.86813, 8.1583, 8.39839, 8.58653, 8.72187, 8.80331, 8.83059, 8.80331], [5.82362, 5.81111, 5.82362, 5.76904, 5.68826, 5.55393, 5.40641, 5.20538, 4.99035, 4.72656  …  5.19038, 4.61399, 4.00892, 3.37934, 2.72876, 2.06148, 1.38139, 0.692846, 0.0, -0.692846], [1.61114e-5, 0.0, 1.61114e-5, -0.000112722, -8.64607e-5, -0.000145293, -0.000163693, -0.000181994, -0.000143675, -0.00023348  …  -0.000211592, -0.000226676, -0.00023778, -0.00025012, -0.000246062, -0.00025497, -0.000264656, -0.00025937, -0.00025881, -0.00025937], [-0.000301409, -0.000536818, -0.000301409, -0.00034067, -0.000272164, -0.000365503, -0.000241693, -0.000364188, -0.000249077, -0.000293274  …  -0.000161749, -0.000140628, -0.000116576, -0.000103485, -8.42061e-5, -5.63279e-5, -3.61267e-5, -2.09209e-5, 0.0, -2.09209e-5], [1.84165, 1.84165, 1.84121, 1.84131, 1.84127, 1.84124, 1.84128, 1.84124, 1.84127, 1.84127  …  1.8

In [130]:
mz

352-element Array{Float64,1}:
 0.04614264014506791
 0.04614264014506791
 0.13814343571472562
 0.22929253106584524
 0.319027961715481  
 0.406796478893486  
 0.4920569605056729 
 0.5742837473307184 
 0.6529698838847235 
 0.7276302439700976 
 0.7978045216398406 
 0.8630600691367585 
 0.9229945643106534 
 ⋮                  
 1.2120242840922826 
 1.3111604836153625 
 1.4022129427521652 
 1.484620292813463  
 1.5578744650838738 
 1.6215238232307172 
 1.6751759477961297 
 1.7185000555907277 
 1.7512290390833802 
 1.7731611132074707 
 1.7841610594325286 
 1.7841610594325286 

In [131]:
var.z

391-element Array{Float64,1}:
  7.9661794817023255
  7.990815209918216 
  7.966179481702274 
  7.892432680196646 
  7.770025995318328 
  7.599714587226309 
  7.382548396232152 
  7.119866340540719 
  6.8132879411908265
  6.464703356038796 
  6.076261724964651 
  5.650357919794039 
  5.189617779113235 
  ⋮                 
  7.065849315989937 
  6.489687512774953 
  5.87351462800361  
  5.221129572408895 
  4.536554516797229 
  3.82401009405573  
  3.087889377076412 
  2.33273080343373  
  1.5631901708017963
  0.7840117264312568
  0.0               
 -0.784011726431261 

In [100]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [208]:
tmp =0.0
for i in 1:21
   tmp += √(var.r[i]^2 + var.z[i]^2)
end
@show tmp/21

tmp / 21 = 4.99588249535343


4.99588249535343

In [209]:
tmax =0.0
for i in 1:21
    tmp2 = √(var.r[i]^2 + var.z[i]^2)
    if abs(tmp2 - tmp/21) >tmax
        tmax = abs(tmp2-tmp/21)
    end
end
@show tmax

tmax = 3.597122599785507e-13


3.597122599785507e-13

In [ ]:
 rcell = zeros(Float64,4)
    zcell = zeros(Float64,4)
    ucell = zeros(Float64,4)
    vcell = zeros(Float64,4)

Ic, = size(Tp) 
for i in 1:Ic
        for j =1:4
            rcell[j] = var.r[Tp[i,j]]
            zcell[j] = var.z[Tp[i,j]]
            ucell[j] = var.u[Tp[i,j]]
            vcell[j] = var.v[Tp[i,j]]
        end
        Az = sum(A_z(rcell,zcell))
@show Az
end

In [141]:
 function Impact()

    global problem = prb(5000,0.1)
    # global problem = prb(5e-5,0.3)
    
    I = 100
    J = 20
    dx = 500/I
    dy = 100/J
    
    global   con1 = Const(3e-3,2.785,2.0,2.7e-1,0.5328,1.338)
    
    I1 = I +1
    J1 = J
    
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1,4) #边界 
    
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    
# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 2
            Tb[ib,4] = 2
        end
    
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 2
             Tb[ib,4] = 1
         end
      for i = 1:I1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 1
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 1
                end
            end
        end
        
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            x[ip] = (i-2)*dx
            y[ip] = (j-1)*dy
            u[ip] = -1.50e-2
            v[ip] = 0.0
        end
    end
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y,u,v = bound_ghost_coordinate!(x,y,u,v)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A)
                 p[ic] = 0
                ρ[ic] = 2.785
                mz[ic] = ρ[ic] * volume(xz,yz)
                s[ic,:] .= 0
        end
    end 
 return var,mz,I1,J1
end

Impact (generic function with 1 method)

In [140]:
var,mz,I,J = Impact()
var = TimeSolve(var,mz)

2020-element Array{Float64,1}:
 -0.0                  
  5.6811364341009865e-5
  8.94579452137183e-6  
  4.029106329760543e-6 
  2.4031118717755225e-6
  1.6380684112609053e-6
  1.2076781871018609e-6
  9.37601765543197e-7  
  7.55104336729152e-7  
  6.250001129520016e-7 
  5.284034968987066e-7 
  4.543628058484315e-7 
  3.961349153478158e-7 
  ⋮                    
  1.844925892379993e-8 
  1.814076318009342e-8 
  1.7840770121615733e-8
  1.754895523612263e-8 
  1.72650097635895e-8  
  1.698863977163236e-8 
  1.671956529438697e-8 
  1.6457519529873262e-8
  1.6202248091308942e-8
  1.595350830823123e-8 
  1.5711068573641863e-8
  1.5474707733713628e-8

In [196]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [75]:
var.v

572-element Array{Float64,1}:
 -1.4633934829493118e-15
 -1.0345411112180169e-15
 -1.4633934829493118e-15
 -8.402729866514592e-16 
 -1.819951438562635e-15 
 -1.6524145564511796e-15
 -1.0847136930431504e-15
 -1.8534701447445326e-15
 -1.2730300809946046e-15
 -1.5045826999307195e-15
 -1.852448122257305e-15 
  1.590841104610437e-16 
  1.277873532391558e-16 
  ⋮                     
  1.4916201984353308e-15
  4.785147484741018e-15 
 -1.2136095546182232e-14
  8.73102430927251e-15  
 -7.300128622214157e-15 
 -3.156780179231275e-15 
  1.184636289854388e-14 
 -1.1481352315820308e-15
 -4.29977164197353e-15  
 -6.678509989591749e-14 
 -4.0995749000291245e-14
 -3.595126613051837e-18 

In [305]:
  function Collapse_cylindrical()

    global problem = prb(70,0.1)
  #  global problem = prb(50,0.3)
    # global problem = prb(5e-5,0.3)
    
    I = 30
    J = 25
    dr = 2.0/J
    dθ = 90/I
    
    I1 = I +2
    J1 = J
    
  
    global   con1 = Const(3.3e-3,1.845,1.16,1.51,0.7998,1.124)
    
 #  global   con1 = Const(3.3e-3,1.845,1.16,0.0,0.7998,1.124)
    Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
     Ip = (I1+1)*(J1+1) # Number of  points
    Ic = I1*J1  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, 2I1 + 2J1,4) #边界 
    
    global  ν = 1
    ## Tb[i,1] ghost cell number
    ## Tb[i,2] bound cell number
    ## Tb[i,3] ghost cell point share with the bound 
    ## Tb[i,4] type of the boundary
    
    
    
   # 点格表，格点表初始化 
    for i in 1:I1 
        for j in 1:J1
            ic = i+(j-1)*I1
            Tp[ic,1] = i + (j-1)*(I1+1)
            Tp[ic,2] = i+1 +(j-1)*(I1+1)
            Tp[ic,3]=i+1+j*(I1+1)
            Tp[ic,4]=i+j*(I1+1) 
        end
    end
    
    for i = 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            Tc[ip,1] = i +(j-1)*I1 #(i,j)
            Tc[ip,2] = i-1+(j-1)*I1 #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I1 #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I1   #(i,j-1)
        end
    end
    
    for i in 1:Ip
        for j in 1:4
            if Tc[i,j]<0 || Tc[i,j]>Ic
                Tc[i,j]=0
            end
        end
    end

# Bound 
  ib = 0
        for j = 1:J1
            i=1
            ib += 1
            ic =i+(j-1)*I1 
            Tb[ib,1] = ic
            Tb[ib,2] = (i+1)+(j-1)*I1
            Tb[ib,3] = 2
            Tb[ib,4] = 2
        end
         for j = 1:J1
             i = I1
            ib += 1
            ic =i+(j-1)*I1
            Tb[ib,1] = ic
             Tb[ib,2] = (i-1)+(j-1)*I1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
             Tb[ib,3] = 4
             Tb[ib,4] = 2
         end
      for i = 1:I1
            for j in (1,J1)
                ib += 1
                ic =i+(j-1)*I1
                
                Tb[ib,1] = ic
               
                if j==1 
                    Tb[ib,2] = i+(j+1-1)*I1
                    Tb[ib,3] = 1
                    Tb[ib,4] = 1
                end
                if j==J1
                    Tb[ib,2] = i+(j-1-1)*I1
                    Tb[ib,3] = 3
                    Tb[ib,4] = 1
                end
            end
        end
        
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    s = zeros(Float64,Ic,3)
    
    for i in 1:I1+1
        for j = 1:J1+1
            ip =i+(j-1)*(I1+1)
            r₁ = (j-1)*dr+8
            θ = 90-(i-2)*dθ
            v₀ = 6.734e-2
            x[ip] = cosd(θ)*r₁
            y[ip] = sind(θ)*r₁
            u[ip] = -v₀*(8/r₁)^2 *cosd(θ)
            v[ip] = -v₀*(8/r₁)^2 *sind(θ)
        end
    end
      
    
    var = Var(x,y,u,v,ρ,p,s)
    x,y,u,v = bound_ghost_coordinate!(x,y,u,v)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I1
        for j in 1:J1
            ic = i+(j-1)*(I1)
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
           # A = A_z(xz,yz)
    
           # @show V, A*xc
            xc = sum(xz)/4
            #  @show V, sum(A)*xc,xc
                p[ic] = 0.0
                ρ[ic] = 1.845
                mz[ic] = ρ[ic]*volume(xz,yz)
                s[ic,:] .= 0    
           # @show A, V
        end
    end 
    
     Ib, = size(Tb)
     for i in 1:Ib
        if Tb[i,4] == 2
            ic = Tb[i,1]
            ic2= Tb[i,2]
            mz[ic] = mz[ic2]
          end 
    end
    return var,mz,I1,J1
end

Collapse_cylindrical (generic function with 1 method)

In [338]:
var,mz,I,J = Collapse_cylindrical()
var,ρAp,rc = TimeSolve(var,mz)

(Fxn[1, 2] + Fxn[2, 1], Fxn[1, 3] + Fxn[2, 4], rhsu) = (0.0, 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

(Var([-0.418647, 0.0, 0.418647, 0.836146, 1.25135, 1.66313, 2.07035, 2.4719, 2.86666, 3.25358  …  9.135, 9.33534, 9.51009, 9.65878, 9.78099, 9.87639, 9.94472, 9.9858, 10.0, 9.9858], [7.98826, 7.99922, 7.98826, 7.9554, 7.90074, 7.82442, 7.72665, 7.60771, 7.46792, 7.30765  …  4.06716, 3.5835, 3.09002, 2.58806, 2.07901, 1.56427, 1.04523, 0.523333, 0.0, -0.523333], [-0.00352311, 0.0, -0.00352311, -0.00703656, -0.0105307, -0.013996, -0.017423, -0.0208021, -0.0241243, -0.0273804  …  -0.0393825, -0.0402462, -0.0409996, -0.0416406, -0.0421674, -0.0425787, -0.0428733, -0.0430438, 0.0, -0.0430438], [0.0672249, -0.0673398, -0.0672249, -0.0669484, -0.0664884, -0.0658461, -0.0650234, -0.0640224, -0.062846, -0.0614973  …  -0.0175342, -0.0154491, -0.0133216, -0.0111576, -0.00896297, -0.00674381, -0.00450617, -0.0022278, -7.55291e-7, 0.0022278], [1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845  …  1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845, 1.845], [8.09534e-8, 

In [340]:
var.u.^2 +var.v.^2

858-element Array{Float64,1}:
 0.004531598877906816 
 0.004534649731255336 
 0.004531598877906816 
 0.004531598877898345 
 0.004531598877897088 
 0.004531598877896712 
 0.004531598877896545 
 0.004531598877896435 
 0.004531598877896373 
 0.004531598877896372 
 0.004531598877896341 
 0.00453159887789633  
 0.004531598877896372 
 ⋮                    
 0.0018584276684544038
 0.0018584276684543728
 0.001858427668454369 
 0.001858427668454384 
 0.0018584276684543947
 0.001858427668454441 
 0.0018584276684544525
 0.0018584276684543853
 0.0018584276684543816
 0.0018577303665675408
 5.704652235740307e-13
 0.0018577303665675408

In [202]:
using DelimitedFiles
Output_point(I,J,var,mz)

In [40]:
function volume(r,z)
    vol= abs(1/6*((r[1]^2-r[3]^2)*(z[2]-z[4])+(r[4]^2 -r[2]^2)*(z[1]-z[3])
        +(r[3]*z[3]-r[1]*z[1])*(r[2]-r[4])+(r[4]*z[4]-r[2]*z[2])*(r[3]-r[1])))
return vol
end

volume (generic function with 1 method)

In [245]:
tmp =0.0
for i in 2:31
   tmp += √(var.r[i]^2 + var.z[i]^2)
end
@show tmp/30

tmax =0.0
for i in 2:31
    tmp2 = √(var.r[i]^2 + var.z[i]^2)
    if abs(tmp2 - tmp/30) >tmax
        tmax = abs(tmp2-tmp/30)
    end
end
@show tmax

tmp / 30 = 7.998831438659208
tmax = 4.739879599924279e-10


4.739879599924279e-10